# ライントレース+物体検知

## 物体検知側の下準備

In [1]:
from jetbot import ObjectDetector

# 物体検知モデルの読み込み
model_od = ObjectDetector('../note_wavashare/object_following/ssd_mobilenet_v2_coco.engine')

In [2]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)  # カメラの起動

In [3]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

mean_obj = 255.0 * np.array([0.485, 0.456, 0.406])
stdev_obj = 255.0 * np.array([0.229, 0.224, 0.225])
normalize_obj = torchvision.transforms.Normalize(mean_obj, stdev_obj)

def preprocess_obj(camera_value):
    global device, normalize_obj
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

## ライントレース側の下準備

In [4]:
# インポート
import torchvision
import torch

# 自分で作成したライントレースのモデルを読み込む
model_rf = torchvision.models.resnet18(pretrained=False)
model_rf.fc = torch.nn.Linear(512, 2)
model_rf.load_state_dict(torch.load('../note_nvidia/road_following/best_steering_models/[nvidia_circuit-ver1.2]_res18.pth'))

<All keys matched successfully>

In [5]:
# GPU側に転送
device = torch.device('cuda')
model_rf = model_rf.to(device)
model_rf = model_rf.eval().half()

In [6]:
# カメラから読み込む映像への前処理を実装
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [7]:
# カメラからの映像をセルの下に出力
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

# camera = Camera()
# image_widget = ipywidgets.Image()
# traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# display(image_widget)

In [8]:
# 駆動系周りの準備
from jetbot import Robot

robot = Robot()  # 駆動系を制御できるモジュールのインポート

# 制御するバーの設定
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

# 描画
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

# 自Jetbotにおいては、以下の設定だと良い感じに動く
# speed gain = 0.1
# steering gain = 0.05
# steering kd = 0
# steering gain = 0

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.05, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [9]:
# 現在の動作を示すバーを描画
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

## 本処理

In [12]:
from jetbot import bgr8_to_jpeg
import time

blocked_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = ipywidgets.Image(format='jpeg', width=300, height=300)
label_widget = ipywidgets.IntText(value=1, description='tracked label')
speed_widget = ipywidgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = ipywidgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(ipywidgets.VBox([
    ipywidgets.HBox([image_widget, blocked_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

angle = 0.0
angle_last = 0.0

def execute(change):
    
    start_time = time.time()
    
    global angle, angle_last
    image = change['new']  # 映像読み込み
    #----------------------------ライントレース部分--------------------------------------------------#
    xy = model_rf(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    #----------------------------------------------------------------------------------------------------#
    
    #--------------------------------------------物体検知部分------------------------------------------------#
    # カメラの映像内において、検知出来た全ての物体を、変数detectionに入れる
    detections = model_od(image)
    
    # 出力されているカメラ映像の上に、物体の枠(バウンディングボックス)を描画する
    label_id=[]
    for det in detections[0]:
        # バウンディングボックスの描画
        bbox = det['bbox']
        image = cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
        
        # ラベルIDを表示
        # label_id = det['label'] と記述してしまうと、det['label']が空の場合、label_idは宣言されない扱いになってしまうので、Noneが代入された配列として宣言する
        label_id.append(str(det['label']))
        
        # {この辺に、「ラベルIDと一致する名前を持ってくる」みたいな処理}
        cv2.putText(image, "id:"+str(det['label']), (int(width * bbox[0]), int(height * bbox[1]+3)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (36,255,12), 2)
        
    # print(label_id)
    
    # select detections that match selected class label
      # 追跡する物体を設定する(ユーザがlabel番号を指定、その番号をmatching_detectionsに代入)
      # label_widget.value : ユーザが指定するやつ、1とか2とか設定できるやつ
      # d['label'] : 
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    # 検知した物体によって動きを変える
    ''' 
    pause_flag = str(44) in label_id  # bottleを検知したら一時停止
    if(pause_flag == True):
        print("\r"+"pause",end="")
        robot.stop()
        time.sleep(0.01)
    '''    
    
    # 出力されているカメラ映像の更新
    image_widget.value = bgr8_to_jpeg(image)
    
    finish_time = time.time()
    excute_time = finish_time - start_time
    print(excute_time)
    
execute({'new': camera.value})
camera.observe(execute, names='value')

0.9410223960876465
0.262157678604126
0.24971318244934082
0.23561501502990723
0.208906888961792
0.23117280006408691
0.23270249366760254
0.24494028091430664
0.20804572105407715
0.2124347686767578
0.20993709564208984
0.2124617099761963
0.23207879066467285
0.2409508228302002
0.2051410675048828
0.21487092971801758
0.19976544380187988
0.2062065601348877
0.19904756546020508
0.21723437309265137
0.2443842887878418
0.21610403060913086
0.20500802993774414
0.20632004737854004
0.20048904418945312
0.20938396453857422
0.21394991874694824
0.23758411407470703
0.20163822174072266
0.2097184658050537
0.20604848861694336
0.21158337593078613
0.20606160163879395
0.2232952117919922
0.23497986793518066
0.21328020095825195
0.21727848052978516
0.20018386840820312
0.2070755958557129
0.20179247856140137
0.22095370292663574
0.23655056953430176
0.2065293788909912
0.20442724227905273
0.20080184936523438
0.20339465141296387
0.20948004722595215
0.21593999862670898


Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [10]:
# camera.observe(execute, names='value')

In [13]:
camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
camera.stop()
robot.stop()

0.23315882682800293
0.21148347854614258


In [13]:
camera.stop()
robot.stop()